In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from utils_datetime import *
from utils_filter import *

In [ ]:
contingencies = xr.load_dataset('data/contingency/contingency2.nc')

In [ ]:
pph = xr.load_dataset('data/pph/labelled2023_pph_final.nc')

In [ ]:
missing_dates = ['200204190000', '200204200000', '200204210000', '200204250000', '200205060000', '200205250000', '200207310000', '200208130000', '200208300000', '200211090000', '200212230000', '200302030000', '200303250000', '200304140000', '200304150000', '200304160000', '200305100000', '200306250000', '200306280000', '200307270000', '200307280000', '200309030000', '200312280000', '200404020000', '200404140000', '200405230000', '200408090000', '200410140000', '200503300000', '200506060000', '200508030000', '200701040000', '200905280000', '201105210000', '202005240000', '202106130000']
pph = pph.sel(time=~pph['time'].isin(missing_dates))
contingencies = contingencies.sel(time=~contingencies['time'].isin(missing_dates))

In [ ]:
# Adapted from https://gist.github.com/djgagne/64516e3ea268ec31fb34

def performance_diagram(pods, fars, obj_labels, colors, markers, filename = '', save = False, show = True, figsize=(9, 8), points = True, xlabel="Success Ratio (1-FAR)",
                        ylabel="Probability of Detection", ticks=np.arange(0, .6, 0.1), dpi=300, csi_cmap="Blues",
                        csi_label="Critical Success Index", title="Performance Diagram",
                        legend_params=dict(loc=4, fontsize=12, framealpha=1, frameon=True)):
    """
    Draws a performance diagram from a set of POD lists and FAR lists.
    :param pods: list or array of POD arrays.
    :param fars: list or array of FAR arrays.
    :param obj_labels: list or array of labels describing each DistributedROC object.
    :param colors: list of color strings
    :param markers: list of markers.
    :param filename: output filename.
    :param save: whether the figure will be saved
    :param show: whether the figure will be shown in the console
    :param figsize: tuple with size of the figure in inches.
    :param points: True if the plotted points should be separate, otherwise connected
    :param xlabel: Label for the x-axis
    :param ylabel: Label for the y-axis
    :param ticks: Array of ticks used for x and y axes
    :param dpi: DPI of the output image
    :param csi_cmap: Colormap used for the CSI contours
    :param csi_label: Label for the CSI colorbar
    :return:
    """
    plt.figure(figsize=figsize)
    grid_ticks = np.arange(0, .51, 0.01)
    sr_g, pod_g = np.meshgrid(grid_ticks, grid_ticks)
    bias = pod_g / sr_g
    csi = 1.0 / (1.0 / sr_g + 1.0 / pod_g - 1.0)
    csi_contour = plt.contourf(sr_g, pod_g, csi, np.arange(.025, .325, 0.025), extend="max", cmap=csi_cmap)
    b_contour = plt.contour(sr_g, pod_g, bias, [.25, 0.5, 1, 2, 4], colors="k", linestyles="dashed")
    plt.clabel(b_contour, fmt="%1.2f", manual=[(0.1, 0.45), (0.2, 0.45), (0.35, 0.35), (.45, .2), (.45, .1)])
    for r, pod, far in zip(range(len(pods)), pods, fars):
        if points:
            plt.scatter(1 - far, pod, marker=markers[r], color=colors[r], label=obj_labels[r], s = 100)
        else:
            plt.plot(1 - far, pod, marker=markers[r], color=colors[r], label=obj_labels[r], markersize = 5, linewidth = 1)
            shift0 = .03
            shift1 = .03
            if r == 3:
                shift1 = 0
            plt.annotate('2002', xy = (1-far[1], pod[1]), xytext=(1-far[1]-shift1, pod[1]+shift0), arrowprops=dict(arrowstyle="->"))
            plt.annotate('2023', xy = (1-far[-1], pod[-1]), xytext=(1-far[-1]+shift0, pod[-1]-shift1), arrowprops=dict(arrowstyle="->"))
    cbar = plt.colorbar(csi_contour)
    cbar.set_label(csi_label, fontsize=14)
    plt.xlabel(xlabel, fontsize=14)
    plt.ylabel(ylabel, fontsize=14)
    plt.xticks(ticks)
    plt.yticks(ticks)
    plt.title(title, fontsize=14, fontweight="bold")
    plt.text(0.24,0.26,"Frequency Bias",fontdict=dict(fontsize=14, rotation=48))

    if len(obj_labels) > 8:
        legend_params=dict(loc='center left', bbox_to_anchor=(1.3, 0.5), fontsize=12, framealpha=1, frameon=True)
    
    plt.legend(**legend_params)

    if save:
        plt.savefig(filename, dpi=dpi, bbox_inches="tight")
    if show:
        plt.show()




In [ ]:
day3_dates = pph['time'][pph['time'] >= '200203300000']
contingencies = contingencies.sel(time = day3_dates)
pph = pph.sel(time = day3_dates)
mdt_dates = pph['time'][pph['MAX_CAT'].isin(['MDT', 'HIGH'])]
mdt_contingencies = contingencies.sel(time = mdt_dates)

high_dates = pph['time'][pph['MAX_CAT'].isin(['HIGH'])]
high_contingencies = contingencies.sel(time = high_dates)

pods = []
fars = []
labels = []
colors = []
markers = []
hazard_types= ['Wind', 'Hail', 'Tornado', 'All Hazard']

color_dict = {
    'Wind': 'dodgerblue',
    'Hail': 'palegreen',
    'Tornado': 'lightcoral',
    'All Hazard': 'lightgray'
}

mdt_color_dict = {
    'Wind': 'navy',
    'Hail': 'darkgreen',
    'Tornado': 'maroon',
    'All Hazard': 'black'
}

color_dict = {
    'Wind': 'blue',
    'Hail': 'green',
    'Tornado': 'red',
    'All Hazard': 'black'
}

mdt_color_dict = {
    'Wind': 'blue',
    'Hail': 'green',
    'Tornado': 'red',
    'All Hazard': 'black'
}

for contingencies_dataset, thresh in zip([contingencies, mdt_contingencies, high_contingencies], ['all', 'mdt', 'high']):
    for hazard in hazard_types:
        c = contingencies_dataset.sel(hazard = hazard)
        
        pods.append(np.array([c['a'].values.sum()/(c['a'].values.sum() + c['c'].values.sum())]))
        fars.append(np.array([c['b'].values.sum()/(c['a'].values.sum() + c['b'].values.sum())]))
        if thresh == 'mdt':
            labels.append(hazard + ' (MDT+)')
            colors.append(mdt_color_dict[hazard])
            markers.append('s')
        elif thresh == 'high':
            labels.append(hazard + ' (HIGH)')
            colors.append(mdt_color_dict[hazard])
            markers.append('*')
        else:
            labels.append(hazard + '(All Days)')
            colors.append(color_dict[hazard])
            markers.append('o')

        # No Skill
        #pods.append(np.array([(c['a'].values.sum() + c['c'].values.sum()) / (c['a'].values.sum() + c['b'].values.sum() + c['c'].values.sum() + c['d'].values.sum())]))
        #fars.append(np.array([1 - (c['a'].values.sum() + c['c'].values.sum()) / (c['a'].values.sum() + c['b'].values.sum() + c['c'].values.sum() + c['d'].values.sum())]))
        #if mdt:
        #    labels.append('_nolegend_')
        #    colors.append(mdt_color_dict[hazard])
        #    markers.append('.')
        #else:
        #    labels.append('_nolegend_')
        #    colors.append(color_dict[hazard])
        #   markers.append('.')
        

performance_diagram(pods, fars, labels, colors, markers, title = "Performance Diagram by Hazard Type and Risk Level Days 2002-2022", save = True, filename = 'plots/results/performacne_diagram_all_and_mdt.png')

In [ ]:
date_contingencies = contingencies.sel(time = '201905200000')
date_contingencies['pod'] = date_contingencies['a'] / (date_contingencies['a'] + date_contingencies['c'])

date_contingencies['far'] = date_contingencies['b'] / (date_contingencies['a'] + date_contingencies['b'])

date_contingencies['csi'] = date_contingencies['a'] / (date_contingencies['a'] + date_contingencies['b'] + date_contingencies['c'])

date_contingencies['bias'] = (date_contingencies['a'] + date_contingencies['b']) / (date_contingencies['a'] + date_contingencies['c'])

pods = []
fars = []

for hazard in hazard_types:
    pods.append(date_contingencies['pod'].sel(hazard = hazard))
    fars.append(date_contingencies['far'].sel(hazard = hazard))
performance_diagram(pods, fars, hazard_types, ['blue', 'green', 'red', 'black'], ['o', 'o', 'o', 'o'], points = True, show = False, save = True, title = 'May 20, 2019 Performance Diagram', filename='plots/results/performance_201905200000.png')

In [ ]:
contingencies['time'] = pd.to_datetime(contingencies['time'].values, format='%Y%m%d0000')
year_contingencies = contingencies.rolling(time=365, center=True).sum()
season_contingencies = contingencies.rolling(time=91, center=True).sum()

contingencies['pod'] = contingencies['a'] / (contingencies['a'] + contingencies['c'])
year_contingencies['pod'] = year_contingencies['a'] / (year_contingencies['a'] + year_contingencies['c'])
season_contingencies['pod'] = season_contingencies['a'] / (season_contingencies['a'] + season_contingencies['c'])

contingencies['far'] = contingencies['b'] / (contingencies['a'] + contingencies['b'])
year_contingencies['far'] = year_contingencies['b'] / (year_contingencies['a'] + year_contingencies['b'])
season_contingencies['far'] = season_contingencies['b'] / (season_contingencies['a'] + season_contingencies['b'])

contingencies['csi'] = contingencies['a'] / (contingencies['a'] + contingencies['b'] + contingencies['c'])
year_contingencies['csi'] = year_contingencies['a'] / (year_contingencies['a'] + year_contingencies['b'] + year_contingencies['c'])
season_contingencies['csi'] = season_contingencies['a'] / (season_contingencies['a'] + season_contingencies['b'] + season_contingencies['c'])

contingencies['bias'] = (contingencies['a'] + contingencies['b']) / (contingencies['a'] + contingencies['c'])
year_contingencies['bias'] = (year_contingencies['a'] + year_contingencies['b']) / (year_contingencies['a'] + year_contingencies['c'])
season_contingencies['bias'] = (season_contingencies['a'] + season_contingencies['b']) / (season_contingencies['a'] + season_contingencies['c'])

Performance Diagram Over Years

In [ ]:
pods = []
fars = []
for hazard in hazard_types:
    pods.append(year_contingencies['pod'].sel(hazard = hazard, time=(year_contingencies.time.dt.month == 6) & (year_contingencies.time.dt.day == 1)))
    fars.append(year_contingencies['far'].sel(hazard = hazard, time=(year_contingencies.time.dt.month == 6) & (year_contingencies.time.dt.day == 1)))
performance_diagram(pods, fars, hazard_types, ['blue', 'green', 'red', 'black'], ['o', 'o', 'o', 'o'], points = False, show = False, save = True, title = 'Yearly Perfomance Diagram by Hazard Type, 2002-2023', filename='plots/results/performance_diagram_all_years.png')

In [ ]:
# PD by region
pods = []
fars = []
labels = []
colors = []
markers = []
hazard_types= ['Wind', 'Hail', 'Tornado', 'All Hazard']
regions = ['West', 'Great Plains', 'Midwest', 'South', 'Northeast']

shape_dict = {
    'West': '<',
    'Great Plains': 's', 
    'Midwest': '^', 
    'South': 'v', 
    'Northeast': '>' 
}

for hazard in hazard_types:
    for region in regions:
        c = contingencies.sel(hazard = hazard, time = contingencies['time'][(pph['REGION_M'] == region).values])
        
        pods.append(np.array([c['a'].values.sum()/(c['a'].values.sum() + c['c'].values.sum())]))
        fars.append(np.array([c['b'].values.sum()/(c['a'].values.sum() + c['b'].values.sum())]))

        labels.append(hazard + ' ' + region)
        colors.append(color_dict[hazard])
        markers.append(shape_dict[region])

performance_diagram(pods, fars, labels, colors, markers, title = "Performance by Hazard Type and Region 2002-2023", save = True, filename = 'plots/results/performacne_diagram_region.png')

In [ ]:
# PD by season
fars = []
labels = []
colors = []
markers = []
hazard_types= ['Wind', 'Hail', 'Tornado', 'All Hazard']
seasons = ['Winter', 'Spring', 'Summer', 'Fall']

shape_dict = {
    'Winter': '^',
    'Spring': '<', 
    'Summer': 'v', 
    'Fall': '>', 
}

for hazard in hazard_types:
    for season in seasons:
        c = contingencies.sel(hazard = hazard, time = contingencies['time'][(pph['SEASON'] == season).values])
        
        pods.append(np.array([c['a'].values.sum()/(c['a'].values.sum() + c['c'].values.sum())]))
        fars.append(np.array([c['b'].values.sum()/(c['a'].values.sum() + c['b'].values.sum())]))

        labels.append(hazard + ' ' + season)
        colors.append(color_dict[hazard])
        markers.append(shape_dict[season])

performance_diagram(pods, fars, labels, colors, markers, title = "Performance by Hazard Type and Season 2002-2023", save = True, filename = 'plots/results/performacne_diagram_season.png')

# Plotting POD, FAR, CSI, Bias timeseries

In [ ]:
def plot_timeseries_all_hazard(contingencies, var, ylabel, show = True, save = False, path = None):
    hazards = ['All Hazard', 'Wind', 'Hail', 'Tornado']
    for hazard, c in zip(hazards, ['black', 'blue', 'green', 'red']):
        plt.plot(contingencies['time'], contingencies[var].sel(hazard = hazard), c = c)
    if var == 'bias':
        plt.yscale('log')
        plt.axhline(y = 1, color = 'black', linestyle = '--') 
        plt.yticks([0.25, 0.5, 1, 2, 4], labels=['0.25', '0.5', '1', '2', '4'])
        plt.minorticks_off()
    plt.xlabel('Time')
    plt.ylabel(ylabel)
    plt.title("Yearly " + ylabel)
    plt.legend(hazards)
    
    if save:
        plt.savefig(path)
    if show:
        plt.show()
    plt.clf()
    
def plot_timeseries_rolling(contingencies1, contingencies2, var, hazard, ylabel, show = True, save = False, path = None):
    plt.plot(contingencies2['time'], contingencies2[var].sel(hazard = hazard), c = 'grey')
    plt.plot(contingencies1['time'], contingencies1[var].sel(hazard = hazard), c = 'black')
    
    if var == 'bias':
        plt.yscale('log')
        plt.axhline(y = 1, color = 'black', linestyle = '--') 
        plt.yticks([0.25, 0.5, 1, 2, 4], labels=['0.25', '0.5', '1', '2', '4'])
        plt.minorticks_off()

    plt.xlabel('Time')
    plt.ylabel(ylabel)
    plt.title('Yearly and Seasonal ' + hazard + ' ' + ylabel)
    plt.legend(['365-Day', '91-Day'])


    if save:
        plt.savefig(path)
    if show:
        plt.show()
    
    plt.clf()

In [ ]:
hazards = ['All Hazard', 'Wind', 'Hail', 'Tornado']
for var, label in zip(['pod', 'far', 'csi', 'bias'], ['Probability of Detection', 'False Alarm Ratio', 'Critical Success Index', 'Bias']):
    plot_timeseries_all_hazard(year_contingencies, var, label, show = False, save = True, path = 'plots/results/timeseries/' + var + '.png')
    for hazard in hazards:
        plot_timeseries_rolling(year_contingencies, season_contingencies, var, hazard, label, show = False, save = True, path = 'plots/results/timeseries/' + var + '_' + hazard + '.png')

# Testing

In [ ]:
pods = [np.empty(4), np.empty(4), np.empty(4), np.empty(4)]
fars = [np.empty(4), np.empty(4), np.empty(4), np.empty(4)]
for i, p in enumerate([.05, .2, .5, .8]):
    for j, v in enumerate([.05, .2, .5, .8]):
        a = p * v
        b = p * (1-v)
        c = (1-p) * v
        pods[j][i] = a / (a + c)
        fars[j][i] = b / (a + b)

performance_diagram(pods, fars, ['True 5', 'True 10', 'True 20', 'True 50'], ['blue', 'green', 'red', 'black'], ['o', 'o', 'o', 'o'], points=False)

In [ ]:
pods = [np.empty(4), np.empty(4), np.empty(4), np.empty(4)]
fars = [np.empty(4), np.empty(4), np.empty(4), np.empty(4)]
for i, p in enumerate([0, .25, .5, .75]):
    for j, v in enumerate([0, .25, .5, .75]):
        a = p * .5
        b = p * .5 + v * 1
        if b == 0:
            b = 1
        c = (1-p) * .5
        pods[j][i] = a / (a + c)
        fars[j][i] = b / (a + b)

performance_diagram(pods, fars, ['Fcst 0 in 0', 'Fcst .25 in 0', 'Fcst .5 in 0', 'Fcst .75 in 0'], ['blue', 'green', 'red', 'black'], ['o', 'o', 'o', 'o'])

In [ ]:
# no skill forecast test
p = np.array([0, .1, .1, .0, .6, .3, 0, .05])

pods = []
fars = []

for f in [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1]:
    a = (f * p).sum()
    b = (f * (1-p)).sum()
    c = ((1-f) * p).sum()
    pods.append(a / (a + c))
    fars.append(b / (a + b))

fars
#1-np.mean(p)
